In [1]:
import os
import sys
import json
import tensorflow as tf
import numpy as np
import pandas as pd

src_dir = os.path.join("../src")
sys.path.insert(0, src_dir)

from model.vtn import VTNBuilder

In [2]:
model = VTNBuilder()
test = tf.random.uniform((1, 3, 15, 224, 224))   # B, C, F, H, W
frame_idx = np.arange(0, 1*15).reshape((1, 15))
out = model([test, frame_idx])
print(out.shape)

2021-07-16 21:01:37.598415: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


The following Variables were used a Lambda layer's call (tf.broadcast_to), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 1, 384) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got c

In [3]:
class WarmupSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(WarmupSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [4]:
learning_rate = WarmupSchedule(128)
optimizer = tf.keras.optimizers.Adam(learning_rate)

In [5]:
with open("../configs/vtn.json", "r") as f:
    config = json.load(f)

In [6]:
arch = type(model).__name__
ckpt = tf.train.Checkpoint(arch=tf.Variable(arch),
                            epoch=tf.Variable(1),
                            model=model,
                            optimizer=optimizer,
                            monitor_best=tf.Variable(0.0),
                            config=tf.Variable(json.dumps(config)))
ckpt.write("../saved/test")

'../saved/test'

In [7]:
arch = tf.Variable("")
epoch = tf.Variable(1)
monitor_best = tf.Variable(0.0)
config = tf.Variable("")
ckpt = tf.train.Checkpoint(arch=arch,
                            epoch=epoch,
                            model=model,
                            optimizer=optimizer,
                            monitor_best=monitor_best,
                            config=config)
ckpt.restore("../saved/test")

In [9]:
int(epoch)

1

In [12]:
float(monitor_best)

0.0

In [11]:
json.loads(config.numpy().decode("utf-8"))

{'name': 'DP_Posture',
 'n_gpu': 1,
 'save_dir': 'saved/',
 'arch': {'type': 'VTNBuilder',
  'args': {'backbone': 'EfficientNetB0', 'n_classes': 16}},
 'data_loader': {'type': 'FrameDataLoaderTF',
  'args': {'data_path': 'data/anonymisedVideos',
   'batch_size': 3,
   'shuffle': True,
   'validation_split': 0.1,
   'resolution': [224, 224],
   'batch_frame': 15,
   'batch_second': 5,
   'num_clips': 12,
   'resize_fac': [0.8, 1.2],
   'mean_norm': [0.45, 0.45, 0.45],
   'std_norm': [0.225, 0.225, 0.225]}},
 'trainer': {'epochs': 40,
  'save_period': 10,
  'verbosity': 2,
  'monitor': 'max val_accuracy',
  'early_stop': 5,
  'lr_dmodel': 128,
  'tensorboard': True}}